### A Seq-Seq model based implementation for Machine Translation (https://arxiv.org/pdf/1409.3215.pdf)
Dataset: Eng-French

In [35]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense, CuDNNLSTM, Bidirectional
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import numpy as np
from nltk.translate.bleu_score import corpus_bleu
import matplotlib.pyplot as plt
import re
from unicodedata import normalize
import string
import pickle
import random

In [24]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra.txt'

In [25]:
def cleanLine(line):
    '''Function to clean the eng-german keyword pair list and removing unprintable chars'''
    # Regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    table = str.maketrans('', '', string.punctuation)
    line = normalize('NFD', line).encode('ascii', 'ignore')
    line = line.decode('UTF-8')
    line = line.split()
    line = [word.lower() for word in line]
    line = [word.translate(table) for word in line]
    line = [re_print.sub('', word) for word in line]
    # Remove numeric chrs
    line = [w for w in line if w.isalpha()]
    return " ".join(line)

In [26]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
#     input_text, target_text = cleanLine(input_text), cleanLine(target_text)
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [27]:
lines[:10]
target_texts[:10]

['\tVa !\n',
 '\tSalut !\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tQui ?\n',
 '\tÇa alors\u202f!\n',
 '\tAu feu !\n',
 "\tÀ l'aide\u202f!\n",
 '\tSaute.\n',
 '\tÇa suffit\u202f!\n']

In [28]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [29]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [30]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [31]:
print(input_texts[:10])
print(target_texts[:10])

['Go.', 'Hi.', 'Run!', 'Run!', 'Who?', 'Wow!', 'Fire!', 'Help!', 'Jump.', 'Stop!']
['\tVa !\n', '\tSalut !\n', '\tCours\u202f!\n', '\tCourez\u202f!\n', '\tQui ?\n', '\tÇa alors\u202f!\n', '\tAu feu !\n', "\tÀ l'aide\u202f!\n", '\tSaute.\n', '\tÇa suffit\u202f!\n']


In [32]:
len(input_texts)

10000

In [33]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [34]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = CuDNNLSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
print(encoder_outputs.shape)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = CuDNNLSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

(?, 256)


In [13]:
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2, callbacks=[checkpoint])

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 24s 3ms/step - loss: 0.9190 - val_loss: 0.9418

Epoch 00001: val_loss improved from inf to 0.94184, saving model to model.h5


C:\Users\Z0042ZRF\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer cu_dnnlstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'cu_dnnlstm_1/strided_slice_16:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'cu_dnnlstm_1/strided_slice_17:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Epoch 2/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.7302 - val_loss: 0.7643

Epoch 00002: val_loss improved from 0.94184 to 0.76426, saving model to model.h5
Epoch 3/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.6199 - val_loss: 0.6771

Epoch 00003: val_loss improved from 0.76426 to 0.67714, saving model to model.h5
Epoch 4/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.5608 - val_loss: 0.6254

Epoch 00004: val_loss improved from 0.67714 to 0.62544, saving model to model.h5
Epoch 5/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.5193 - val_loss: 0.6050

Epoch 00005: val_loss improved from 0.62544 to 0.60504, saving model to model.h5
Epoch 6/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.4876 - val_loss: 0.5587

Epoch 00006: val_loss improved from 0.60504 to 0.55869, saving model to model.h5
Epoch 7/100
8000/8000 [==============================] - 11s 1ms/step 

8000/8000 [==============================] - 11s 1ms/step - loss: 0.1160 - val_loss: 0.5643

Epoch 00051: val_loss did not improve from 0.44480
Epoch 52/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.1131 - val_loss: 0.5647

Epoch 00052: val_loss did not improve from 0.44480
Epoch 53/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.1107 - val_loss: 0.5722

Epoch 00053: val_loss did not improve from 0.44480
Epoch 54/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.1082 - val_loss: 0.5769

Epoch 00054: val_loss did not improve from 0.44480
Epoch 55/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.1058 - val_loss: 0.5826

Epoch 00055: val_loss did not improve from 0.44480
Epoch 56/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.1039 - val_loss: 0.5839

Epoch 00056: val_loss did not improve from 0.44480
Epoch 57/100
8000/8000 [==============================] - 11s 1ms/step 

In [16]:
model = load_model('model.h5')

In [17]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [22]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    target_seq = decoder_target_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    print('Target sentence:', target_texts[seq_index][1:-1])

-
Input sentence: Go.
Decoded sentence: Va !

Target sentence: Va !
-
Input sentence: Hi.
Decoded sentence: Salut !

Target sentence: Salut !
-
Input sentence: Run!
Decoded sentence: Courez !

Target sentence: Cours !
-
Input sentence: Run!
Decoded sentence: Courez !

Target sentence: Courez !
-
Input sentence: Who?
Decoded sentence: Qui ?

Target sentence: Qui ?
-
Input sentence: Wow!
Decoded sentence: Ça alors !

Target sentence: Ça alors !
-
Input sentence: Fire!
Decoded sentence: At alsez !

Target sentence: Au feu !
-
Input sentence: Help!
Decoded sentence: À l'aide !

Target sentence: À l'aide !
-
Input sentence: Jump.
Decoded sentence: Saute.

Target sentence: Saute.
-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

Target sentence: Ça suffit !
-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

Target sentence: Stop !
-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

Target sentence: Arrête-toi !
-
Input sentence: Wait!
Decoded sentence: Attends !

Target senten

-
Input sentence: Call me.
Decoded sentence: Appellez-moi !

Target sentence: Appelle-moi !
-
Input sentence: Call me.
Decoded sentence: Appellez-moi !

Target sentence: Appellez-moi !
-
Input sentence: Call us.
Decoded sentence: Appelle-nous !

Target sentence: Appelle-nous !
-
Input sentence: Call us.
Decoded sentence: Appelle-nous !

Target sentence: Appelez-nous !
-
Input sentence: Come in.
Decoded sentence: Entrez !

Target sentence: Entrez !
-
Input sentence: Come in.
Decoded sentence: Entrez !

Target sentence: Entre.
-
Input sentence: Come in.
Decoded sentence: Entrez !

Target sentence: Entre !
